<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="img/cover-small.jpg" />

Dieses Notizbuch enthält einen angepassten Auszug aus der [Whirlwind Tour of Python](http://www.oreilly.com/programming/free/a-whirlwind-tour-of-python.csp) von Jake VanderPlas; Der Inhalt ist auf [GitHub](https://github.com/jakevdp/WhirlwindTourOfPython) verfügbar.

Text und Code werden unter der [CC0](https://github.com/jakevdp/WhirlwindTourOfPython/blob/master/LICENSE)- Lizenz veröffentlicht; Das Begleitprojekt, das [Python Data Science Handbook](https://github.com/jakevdp/PythonDataScienceHandbook) wird sehr empfohlen.


# Iteratoren

Ein wichtiger Teil von Datenanalysen besteht oft darin, eine ähnliche Berechnung immer wieder automatisch durchzuführen.
Vielleicht haben wir zum Beispiel eine Tabelle mit Namen, die wir in Vor- und Nachnamen aufteilen möchten, oder mit Daten, die wir in ein Standardformat konvertieren möchten.
Eine der Antworten von Python auf diese Herausforderung ist die *iterator*-Syntax.
Wir haben dies bereits mit dem ``range``-Iterator gesehen:

In [2]:
for i in range(10):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 

Hier werden wir ein bisschen tiefer graben.
Es stellt sich heraus, dass ``range`` in Python 3 keine Liste ist, sondern ein so genannter *Iterator*. Zu verstehen, wie er funktioniert, ist der Schlüssel zum Verständnis einer großen Bandbreite von sehr nützlichen Python-Funktionen.

## Iteration über Listen
Iteratoren sind vielleicht am einfachsten zu verstehen, wenn man über eine Liste iteriert.
Betrachten wir das folgende Beispiel:

In [3]:
for value in [2, 4, 6, 8, 10]:
    # do some operation
    print(value + 1, end=' ')

3 5 7 9 11 

Die vertraute „``for x in y``“-Syntax ermöglicht es uns, eine Operation für jeden Wert in der Liste zu wiederholen.
Die Tatsache, dass die Syntax des Codes so nah an seiner englischen Beschreibung liegt („*für [jeden] Wert in [der] Liste*“), ist nur eine der syntaktischen Entscheidungen, die Python zu einer so intuitiv zu erlernenden und zu verwendenden Sprache machen.

Im Hintegrund passiert einiges mehr. Wenn wir etwas wie „``for val in L``“ schreiben, prüft der Python-Interpreter, ob eine *iterator*-Schnittstelle existiert, was wir selbst mit der eingebauten Funktion ``iter`` überprüfen können:

In [3]:
iter([2, 4, 6, 8, 10])

Es ist dieses Iterator-Objekt, das die von der ``for``-Schleife benötigten Funktionen bereitstellt.
Das ``iter``-Objekt ist ein Container, der uns Zugriff auf das nächste Objekt gibt, solange es existiert, was man mit der eingebauten Funktion ``next`` sehen kann:

In [4]:
I = iter([2, 4, 6, 8, 10])

In [5]:
print(next(I))

2


In [6]:
print(next(I))

4


In [7]:
print(next(I))

6


Was ist der Zweck dieser Indirektion?
Nun, es stellt sich heraus, dass dies unglaublich nützlich ist, weil es Python erlaubt, Dinge als Listen zu behandeln, die *nicht wirklich Listen* sind.

## ``range()``: Eine Liste ist nicht immer eine Liste
Das vielleicht bekannteste Beispiel für diese indirekte Iteration ist die Funktion ``range()`` in Python 3 (in Python 2 ``xrange()`` genannt), die keine Liste, sondern ein spezielles ``range()`` Objekt zurückgibt:

In [8]:
range(10)

range(0, 10)

``range`` stellt, wie eine Liste, einen Iterator bereit:

In [8]:
iter(range(10))

Python weiß also, dass es *wie* eine Liste zu behandeln ist:

In [10]:
for i in range(10):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 

Der Vorteil der Iterator-Indirektion ist, dass *die vollständige Liste nie explizit erstellt wird*!
Wir können dies sehen, indem wir eine `range`-Kalkulation durchführen, die unseren Systemspeicher überwältigen würde, wenn wir sie tatsächlich instanziieren würden (beachten Sie, dass in Python 2 ``range`` eine Liste erzeugt, so dass die Ausführung des Folgenden dort nicht zu guten Resultaten führen wird!):

In [9]:
N = 10 ** 12
for i in range(N):
    if i >= 10: break
    print(i, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

Wenn ``range`` tatsächlich eine Liste von einer Billion Werten erstellen würde, würde sie Dutzende von Terabytes an Maschinenspeicher belegen: eine Verschwendung, wenn man bedenkt, dass wir alle Werte außer den ersten 10 ignorieren!

Tatsächlich gibt es keinen Grund, warum Iteratoren überhaupt jemals enden müssen!
Pythons ``itertools``-Bibliothek enthält eine ``count``-Funktion, die als unendlicher Bereich (`range`) fungiert:

In [12]:
from itertools import count

for i in count():
    if i >= 10:
        break
    print(i, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

Hätten wir hier keine Schleifenunterbrechung (`break`) eingefügt, würde die Zählung munter weitergehen, bis der Prozess manuell unterbrochen oder beendet wird (z.B. mit ``ctrl-C``).

## Nützliche Iteratoren
Diese Iterator-Syntax wird fast universell in den eingebauten Python-Typen sowie in den eher datenwissenschaftlich orientierten Objekten verwendet. Im Folgenden werden wir einige der nützlichsten Iteratoren in Python behandeln:

### ``enumerate``
Oft muss man nicht nur die Werte in einem Array durchlaufen, sondern auch den Index im Auge behalten.
Wir könnten versucht sein, das auf diese Weise zu tun:

In [10]:
L = [2, 4, 6, 8, 10]
for i in range(len(L)):
    print(i, L[i])

0 2
1 4
2 6
3 8
4 10


Obwohl dies funktioniert, bietet Python eine sauberere Syntax mit dem Iterator ``enumerate``:

In [11]:
for i, val in enumerate(L):
    print(i, val)

0 2
1 4
2 6
3 8
4 10


Die entspricht eher dem Python-Weg, Indizes und Werte einer Liste aufzuzählen.

### ``zip``
In anderen Fällen haben wir vielleicht mehrere Listen, über die wir gleichzeitig iterieren wollen.
Wir könnten natürlich über den Index iterieren, wie wir es uns zuvor angesehen haben, aber es ist besser, den Iterator ``zip`` zu verwenden, der *Iterables* zusammenfasst:

In [15]:
L = [2, 4, 6, 8, 10]
R = [3, 6, 9, 12, 15]
for lval, rval in zip(L, R):
    print(lval, rval)

2 3
4 6
6 9
8 12
10 15


Eine beliebige Anzahl von *Iterables* kann zusammengezippt werden. Bei unterschiedlicher Länge entscheidet die kürzeste die Länge des durch `zip` erzeugten *Iterable*.

### ``map`` and ``filter``
Der Iterator ``map`` nimmt eine Funktion und wendet sie auf die Werte in einem Iterator an:

In [16]:
# find the first 10 square numbers
square = lambda x: x ** 2
for val in map(square, range(10)):
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 

Der Iterator ``filter`` sieht ähnlich aus, mit dem Unterschied, dass er nur Werte durchlässt, für die die Filterfunktion den Wert True ergibt:

In [17]:
# find values up to 10 for which x % 2 is zero
is_even = lambda x: x % 2 == 0
for val in filter(is_even, range(10)):
    print(val, end=' ')

0 2 4 6 8 

Die Funktionen ``map`` und ``filter`` sowie die Funktion ``reduce`` (die in Pythons Modul ``functools`` zu finden ist) sind grundlegende Bestandteile des *funktionalen Programmierstils*, der zwar kein dominanter Programmierstil in der Python-Welt ist, aber seine entschiedenen Befürworter hat (siehe z.B. die [pytoolz](https://toolz.readthedocs.org/en/latest/)-Bibliothek).

### Iterators as function arguments

Wir haben gesehen, dass ``*args`` und ``**kwargs`` verwendet werden können, um Sequenzen und Wörterbücher an Funktionen zu übergeben.
Es stellt sich heraus, dass die ``*args``-Syntax nicht nur mit Sequenzen, sondern mit jedem Iterator funktioniert:

In [18]:
print(*range(10))

0 1 2 3 4 5 6 7 8 9


So können wir zum Beispiel trickreich vorgehen und das ``map``-Beispiel von vorhin in das folgende umwandeln:

In [19]:
print(*map(lambda x: x ** 2, range(10)))

0 1 4 9 16 25 36 49 64 81


Mit diesem Trick können wir die uralte Frage beantworten, die in den Foren der Python-Lerner auftaucht: Warum gibt es keine ``unzip()`` Funktion, die das Gegenteil von ``zip()`` macht?
Wer eine Weile darüber nachdenkt, wird vielleicht feststellen, daß das Gegenteil von ``zip()`` ``zip()`` selbst ist! Der Schlüssel ist, daß ``zip()`` eine beliebige Anzahl von Iteratoren oder Sequenzen zusammenzippen kann. Wir sehen:

In [20]:
L1 = (1, 2, 3, 4)
L2 = ('a', 'b', 'c', 'd')

In [21]:
z = zip(L1, L2)
print(*z)

(1, 'a') (2, 'b') (3, 'c') (4, 'd')


In [22]:
z = zip(L1, L2)
new_L1, new_L2 = zip(*z)
print(new_L1, new_L2)

(1, 2, 3, 4) ('a', 'b', 'c', 'd')


Denken Sie eine Weile darüber nach. Wenn Sie verstehen, warum es funktioniert, haben Sie einen großen Schritt zum Verständnis von Python-Iteratoren gemacht!

## Spezialisierte Iteratoren: ``itertools``

Wir haben uns kurz mit dem unendlichen ``range``-Iterator, ``itertools.count``, beschäftigt.
Das Modul ``itertools`` enthält eine ganze Reihe von nützlichen Iteratoren; es lohnt sich, das Modul zu erforschen, um zu sehen, was alles verfügbar ist.
Beispielsweise iteriert die Funktion ``itertools.permutations`` über alle Permutationen einer Folge:

In [23]:
from itertools import permutations
p = permutations(range(3))
print(*p)

(0, 1, 2) (0, 2, 1) (1, 0, 2) (1, 2, 0) (2, 0, 1) (2, 1, 0)


In ähnlicher Weise iteriert die Funktion ``itertools.combinations`` über alle eindeutigen Kombinationen von ``N`` Werten innerhalb einer Liste:

In [24]:
from itertools import combinations
c = combinations(range(4), 2)
print(*c)

(0, 1) (0, 2) (0, 3) (1, 2) (1, 3) (2, 3)


Etwas verwandt ist der ``product``-Iterator, der über alle Mengen von Paaren zwischen zwei oder mehr Iterables iteriert:

In [25]:
from itertools import product
p = product('ab', range(3))
print(*p)

('a', 0) ('a', 1) ('a', 2) ('b', 0) ('b', 1) ('b', 2)


Viele weitere nützliche Iteratoren existieren in ``itertools``: die vollständige Liste kann zusammen mit einigen Beispielen in der [Online-Dokumentation] von Python gefunden werden (https://docs.python.org/3.5/library/itertools.html).